## benchmarking and profilinng
IMPORTANT: benchmark/profile your code!
You can read spec sheets (marketing material) and papers
...but performance depends on your library version, your hardware, your workload
...so there is no substitute for benchmarking/profiling your code.

Example computation: running forward/backward passes on an MLP.
    run_mlp(dim=128, num_layers=16, batch_size=128, num_steps=5)
 - benchmarking()       # How long does it take?
 - profiling()          # Where time is being spent?
    
<mark>Every time you make a change, benchmark/profile!</mark>




-----

## 性能基准测试与多层感知器 (MLP)

### 什么是性能基准测试？

**性能基准测试** (Benchmarking) 是一种衡量执行某项操作所需\*\*“挂钟时间” (wall-clock time)\*\* 的方法。它提供了端到端的时间消耗，但不会告诉你具体时间花在了哪里。要进行更细粒度的分析，需要使用性能剖析 (Profiling) 工具。

性能基准测试在以下场景中非常有用：

  * **比较不同实现**：找出哪种实现方式更快。
  * **理解性能扩展**：了解性能如何随参数（如维度、批量大小）的变化而变化。

-----

### MLP 模型定义

我们使用一个简单的多层感知器 (MLP) 模型进行测试。该模型由一系列线性层和 GeLU 激活函数交替组成。

```python
import torch.nn as nn

class MLP(nn.Module):
    """一个简单的MLP：线性 -> GeLU -> 线性 -> GeLU -> ... -> 线性 -> GeLU"""
    def __init__(self, dim: int, num_layers: int):
        super().__init__()
        self.layers = nn.ModuleList([nn.Linear(dim, dim) for _ in range(num_layers)])

    def forward(self, x: torch.Tensor):
        for layer in self.layers:
            x = layer(x)
            x = torch.nn.functional.gelu(x)
        return x
```

-----

### 基准测试函数

`benchmark` 函数用于对任意可调用函数进行计时。为了获得更准确的结果，它包含了**预热 (warmup)** 阶段和多次试验 (`num_trials`)。预热阶段可以避免由于编译和缓存未命中导致的初始运行时间较慢。

```python
import time
from typing import Callable
from statistics import mean

def benchmark(description: str, run: Callable, num_warmups: int = 1, num_trials: int = 3):
    """
    通过运行 `num_trials` 次来对 `func` 进行基准测试，并返回所有时间。
    - 预热：前几次运行可能会因为编译和缓存问题较慢，我们只关心稳定状态下的时间。
    - 在CUDA可用时，会调用 torch.cuda.synchronize() 来等待所有CUDA线程完成，
      这对于准确计时非常重要。
    """
    # 预热阶段
    for _ in range(num_warmups):
        run()
    if torch.cuda.is_available():
        torch.cuda.synchronize()

    # 实际计时
    times: list[float] = []
    for trial in range(num_trials):
        start_time = time.time()
        run()
        if torch.cuda.is_available():
            torch.cuda.synchronize()
        end_time = time.time()
        times.append((end_time - start_time) * 1000) # 转换为毫秒

    mean_time = mean(times)
    return mean_time
```

-----

### 性能基准测试结果分析

我们对矩阵乘法和 MLP 模型在不同参数下的性能进行了基准测试。

#### 1\. 矩阵乘法

我们测试了不同维度 (`dim`) 下的方阵乘法性能。

| 维度 (dim) | 耗时 (ms) |
| :--- | :--- |
| 1024 | ... |
| 2048 | ... |
| 4096 | ... |
| 8192 | ... |
| 16384 | ... |

**观察**：随着维度增加，矩阵乘法的计算量呈 $O(\\text{dim}^3)$ 增长，因此耗时会显著增加。

#### 2\. MLP 模型性能

我们以一个基准配置作为参考：`dim=256, num_layers=4, batch_size=256, num_steps=2`。

  * **基准 MLP 耗时**：`mlp_base` 记录了此配置下的平均耗时。

-----

##### 性能扩展性分析：

我们分别增加 `num_steps`、`num_layers`、`batch_size` 和 `dim` 来观察性能变化。

**扩展 `num_steps`**：
`num_steps` 是循环运行模型的次数。

  * **结果**：`run_mlp` 的耗时与 `num_steps` 呈**线性关系**。

| 倍数 (scale) | num\_steps | 耗时 (ms) |
| :--- | :--- | :--- |
| 2 | 4 | ... |
| 3 | 6 | ... |
| 4 | 8 | ... |
| 5 | 10 | ... |

**扩展 `num_layers`**：
`num_layers` 决定了 MLP 中线性层和 GeLU 层的数量。

  * **结果**：耗时与 `num_layers` 呈**线性关系**，因为计算量是线性增加的。

| 倍数 (scale) | num\_layers | 耗时 (ms) |
| :--- | :--- | :--- |
| 2 | 8 | ... |
| 3 | 12 | ... |
| 4 | 16 | ... |
| 5 | 20 | ... |

**扩展 `batch_size`**：
`batch_size` 决定了每次前向/反向传播中并行处理的样本数。

  * **结果**：耗时与 `batch_size` 呈**线性关系**。

| 倍数 (scale) | batch\_size | 耗时 (ms) |
| :--- | :--- | :--- |
| 2 | 512 | ... |
| 3 | 768 | ... |
| 4 | 1024 | ... |
| 5 | 1280 | ... |

**扩展 `dim`**：
`dim` 是模型的输入/输出维度，也是线性层中矩阵乘法的维度。

  * 线性层中的矩阵乘法计算量为 $O(\\text{dim}^2)$。
  * **结果**：耗时与 `dim` 呈**平方关系**（或接近），这比其他参数扩展的影响更显著。

| 倍数 (scale) | dim | 耗时 (ms) |
| :--- | :--- | :--- |
| 2 | 512 | ... |
| 3 | 768 | ... |
| 4 | 1024 | ... |
| 5 | 1280 | ... |

-----

### 注意事项

  * 测试结果可能会因 CUDA 内核、硬件等因素而有所波动，因此不总是完全可预测的。
  * `torch.utils.benchmark` 是一个功能更强大的基准测试工具，提供了更多便利。这里为了让过程更透明，我们使用了自定义的简单实现。